In [1]:
import pulp

def steel_slab_pulp():
    model = pulp.LpProblem("Steel_Slabs", pulp.LpMaximize)

    # Define the variables
    slabs_group1 = pulp.LpVariable(name="Slabs1", lowBound=0, cat=pulp.LpContinuous)
    slabs_group2 = pulp.LpVariable(name="Slabs2", lowBound=0, cat=pulp.LpContinuous)
    slabs_group3 = pulp.LpVariable(name="Slabs3", lowBound=0, cat=pulp.LpContinuous)

    # The three constraints are entered for the segments
    model += slabs_group1 <= group1_limit, "Group1"
    model += slabs_group2 <= (group2_limit - group1_limit), "Group2"
    model += slabs_group3 <= (group3_limit - group2_limit), "Group3"
    # # Alternatively, you can write each of the constraints above using pulp's object-oriented (OO)
    # # version and that way, see the similarity with gurobi. I do the first one
    # model.addConstraint(pulp.LpConstraint(e=slabs_group1, sense=pulp.LpConstraintLE, rhs=group1_limit, name='Group 1'))

    # the budget constraint
    budget_constr = group1_costs * slabs_group1 + group2_costs * slabs_group2 + group3_costs * slabs_group3
    model += budget_constr <= budget, "Budget"

    # Objective function
    total_revenue = revenue_per_slab * (slabs_group1 + slabs_group2 + slabs_group3)
    total_cost = group1_costs * slabs_group1 + group2_costs * slabs_group2 + group3_costs * slabs_group3
    # if you pay attention, you see that total_cost expression is the same as budget_constr. 
    # That means you could use that instead. i.e. you can say: 
    # total_profit = total_revenue - budget_constr
    total_profit = total_revenue - total_cost
    model.setObjective(total_profit)    
    model.writeLP(model.name + '.lp')
    
    # Extra: we can pass some parameters to solve() and control the outputs
    # model.solve()  # So rather than this one, we can do the following
    _solver = pulp.PULP_CBC_CMD(msg=False)
    model.solve(solver=_solver)

    print("Status:", pulp.LpStatus[model.status])
    for v in model.variables():
        print(v.name, "=", v.varValue)
    print("Total Profit of Plan = ", pulp.value(model.objective))
#     # If you need extra info such as the slack variables of each constraint
#     for name, constraint in model.constraints.items():
#         print(name, "Remaining Slack ", constraint.slack)

In [2]:
import gurobipy as gp

def steel_slab_gurobi():
    model = gp.Model('Steel_Slabs')

    # Define the variables
    slabs_group1 = model.addVar(vtype=gp.GRB.CONTINUOUS, name='Slabs1')
    slabs_group2 = model.addVar(vtype=gp.GRB.CONTINUOUS, name='Slabs2')
    slabs_group3 = model.addVar(vtype=gp.GRB.CONTINUOUS, name='Slabs3')

    # The three constraints are entered for the segments
    model.addConstr(slabs_group1 <= group1_limit, "Group1")
    model.addConstr(slabs_group2 <= (group2_limit - group1_limit), "Group2")
    model.addConstr(slabs_group3 <= (group3_limit - group2_limit), "Group3")
    # # Alternatively you can write each of the constraints above in the following way
    # # to see their similarity with the object-oriented (OO) version of pulp. I do the first one.
    # model.addConstr(lhs=slabs_group1, sense=gp.GRB.LESS_EQUAL, rhs=group1_limit, name="Group 1")

    # the budget constraint
    budget_constr = group1_costs * slabs_group1 + group2_costs * slabs_group2 + group3_costs * slabs_group3
    model.addConstr(budget_constr <= budget, "Budget")    
    
    # Objective function
    total_revenue = revenue_per_slab * (slabs_group1 + slabs_group2 + slabs_group3)
    total_cost = group1_costs * slabs_group1 + group2_costs * slabs_group2 + group3_costs * slabs_group3
    # if you pay attention, you see that total_cost expression is the same as budget_constr. 
    # That means you could use that instead. i.e. you can say: 
    # total_profit = total_revenue - budget_constr
    total_profit = total_revenue - total_cost
    model.setObjective(total_profit, gp.GRB.MAXIMIZE)
    model.write(model.ModelName + '.lp')
    
    # model.setParam('OutputFlag', 0)
    model.optimize()
    print("Status:", model.status)
    for v in model.getVars():
        print(v.varName, "=", v.x)
    print("Total Profit of Plan = ", model.objVal)
#     # If you need extra info such as the slack variables of each constraint
#     for constraint in model.getConstrs():
#         print(constraint.ConstrName, "Remaining Slack ", constraint.Slack)

In [4]:
if __name__ == '__main__':
    # This is a simple LP for a piecewise equation

    # Define the parameters
    revenue_per_slab = 5.5
    budget = 3000
    group1_limit = 120
    group2_limit = 250
    group3_limit = 99999

    group1_costs = 3
    group2_costs = 4
    group3_costs = 5
#     group1_costs = 5
#     group2_costs = 4
#     group3_costs = 3
    
#     steel_slab_pulp()
    steel_slab_gurobi()

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: 11th Gen Intel(R) Core(TM) i7-1185G7 @ 3.00GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 4 rows, 3 columns and 6 nonzeros
Model fingerprint: 0xbce8ae70
Coefficient statistics:
  Matrix range     [1e+00, 5e+00]
  Objective range  [5e-01, 3e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+02, 1e+05]
Presolve removed 4 rows and 3 columns
Presolve time: 0.01s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.5000000e+03   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.02 seconds (0.00 work units)
Optimal objective  2.500000000e+03
Status: 2
Slabs1 = 0.0
Slabs2 = 0.0
Slabs3 = 1000.0
Total Profit of Plan =  2500.0
